In [2]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from scipy.stats import pearsonr
import scipy.sparse as sp_sparse
import scanpy as sc
from math import log
from statistics import median

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import Model
from tensorflow.keras import layers
from tensorflow.keras.optimizers import Adam
from keras.callbacks import EarlyStopping
from keras import backend as K
import keras.losses

from sklearn import datasets, linear_model
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"


# Data Preparation

In [3]:
adata = sc.read_10x_mtx(
    '/home/ahmadazim/data/filtered_gene_bc_matrices/hg19',  # the directory with the `.mtx` file
    var_names='gene_symbols',                      # use gene symbols for the variable names (variables-axis index)
    cache=True)

adata.var_names_make_unique()

In [4]:
data = pd.DataFrame.sparse.from_spmatrix(adata.X)
print('Working on {} cells and {} genes'.format(*data.shape))

Working on 2700 cells and 32738 genes


In [5]:
# Filter out genes that are not expressed in any cells
geneSum = data.sum(axis=0)
x = geneSum.index[geneSum == 0].tolist()
data = data.drop(x, axis = 1)
data.shape

(2700, 16634)

In [6]:
# Normalizing data (using method from Rao, et al.)
cellSum  = data.sum(axis=1)
median_j = median(cellSum)
npData = np.asarray(data)
for j in range(2700):
    cellSum_j = cellSum[j]
    for i in range(16634):
        npData[j,i] = log( ( (npData[j,i])/(cellSum_j) * median_j ) + 1)

In [7]:
dataNorm = pd.DataFrame(npData)
dataNorm

,0,1,2,3,4,5,6,7,8,9,...,16624,16625,16626,16627,16628,16629,16630,16631,16632,16633
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.532536,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.522734,0.0,0.0,0.370248,0.0,0.000000,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.332558,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.980213,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.175435,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2695,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.491513,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0
2696,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.266796,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0
2697,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.827533,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0
2698,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.145975,0.0,0.0,0.000000,0.0,1.145975,0.0,0.0,0.0


# Implementing Autoencoder model

### (Info from TF tutorial below, but model not actually implemented)
- Use 2 small ConvNets for the encoder and decoder networks (inference/recognition and generative models)
- *x* = observation and *z* = latent variable
- Encoder network defines the approximate posterior distribution *q(z|x)*
    - Use 2 convolutional layers followed by a fully-connected layer
- Decoder network defines the approximate conditional distribution *p(x|z)*
    - Use a fully-connected layer followed by three convolution transpose layers
- Reparameterization Trick...
    - Backpropagation cannot flow through random node
    - Approximate *z* using the decoder parameters and another parameter $\epsilon$ as follows: 
        - $z = \mu + \sigma \odot \epsilon$
        - where $\mu$ and $\sigma$ represent the mean and standard deviation of a Gaussian distribution respectively
    - Enables model to backpropagate gradients in the encoder through $\mu$ and $\sigma$ while maintaining stochasticity through $\epsilon$

## Convolutional Variational AutoEncoder (keras tutorial, but 1 Dimensional)

### Going to try dividing matrix into small "images" 
(2700 x 16634) --> 225 (12 x 72)

In [195]:
VMR = dataNorm.std() / dataNorm.mean() 
lowestVMR = VMR.sort_values(ascending=False)[(225*72):]
fullData = np.asarray(dataNorm.drop(lowestVMR.index, axis= 1))
fullData.shape

(2700, 16200)

In [241]:
# "Chunking" the matrix 
reshapeData = np.array([[[0 for x in range(72)] for x in range(12)] for x in range(50625)], dtype = 'float32')
for i in range(0, 2700, 12):   # i refers to cells (12)
    print("Starting i =", (i/12)+1)
    for j in range(0, 16200, 72):   # j refers to genes (72)
        x = ( 225*(i/12) ) + ( (j/72)+1 )
        smallMat = fullData[ i:(i+12) , j:(j+72) ]
        reshapeData[int(x-1),:,:] = smallMat

reshapeData = reshapeData.reshape((reshapeData.shape[0], reshapeData.shape[1], reshapeData.shape[2], 1))
reshapeData.shape

Starting i = 1.0
Starting i = 2.0
Starting i = 3.0
Starting i = 4.0
Starting i = 5.0
Starting i = 6.0
Starting i = 7.0
Starting i = 8.0
Starting i = 9.0
Starting i = 10.0
Starting i = 11.0
Starting i = 12.0
Starting i = 13.0
Starting i = 14.0
Starting i = 15.0
Starting i = 16.0
Starting i = 17.0
Starting i = 18.0
Starting i = 19.0
Starting i = 20.0
Starting i = 21.0
Starting i = 22.0
Starting i = 23.0
Starting i = 24.0
Starting i = 25.0
Starting i = 26.0
Starting i = 27.0
Starting i = 28.0
Starting i = 29.0
Starting i = 30.0
Starting i = 31.0
Starting i = 32.0
Starting i = 33.0
Starting i = 34.0
Starting i = 35.0
Starting i = 36.0
Starting i = 37.0
Starting i = 38.0
Starting i = 39.0
Starting i = 40.0
Starting i = 41.0
Starting i = 42.0
Starting i = 43.0
Starting i = 44.0
Starting i = 45.0
Starting i = 46.0
Starting i = 47.0
Starting i = 48.0
Starting i = 49.0
Starting i = 50.0
Starting i = 51.0
Starting i = 52.0
Starting i = 53.0
Starting i = 54.0
Starting i = 55.0
Starting i = 56.0
S

(50625, 12, 72, 1)

In [20]:
# Create a sampling layer
class Sampling(layers.Layer):
    """Uses (z_mean, z_log_var) to sample z, the vector encoding a digit."""

    def call(self, inputs):
        z_mean, z_log_var = inputs
        batch = tf.shape(z_mean)[0]
        dim = tf.shape(z_mean)[1]
        epsilon = tf.keras.backend.random_normal(shape=(batch, dim))
        return z_mean + tf.exp(0.5 * z_log_var) * epsilon

In [71]:
# Build the encoder
latent_dim = 32

encoder_inputs = keras.Input(shape=(12, 72, 1))
x = layers.Conv2D(filters= 32, kernel_size= 3, activation="relu", strides=2, padding="same")(encoder_inputs)
x = layers.Conv2D(filters= 64, kernel_size= 3, activation="relu", strides=2, padding="same")(x)
x = layers.Flatten()(x)
x = layers.Dense(16, activation="relu")(x)
z_mean = layers.Dense(latent_dim, name="z_mean")(x)
z_log_var = layers.Dense(latent_dim, name="z_log_var")(x)
z = Sampling()([z_mean, z_log_var])
encoder = keras.Model(encoder_inputs, [z_mean, z_log_var, z], name="encoder")
encoder.summary()

Model: "encoder"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_16 (InputLayer)           [(None, 12, 72, 1)]  0                                            
__________________________________________________________________________________________________
conv2d_14 (Conv2D)              (None, 6, 36, 32)    320         input_16[0][0]                   
__________________________________________________________________________________________________
conv2d_15 (Conv2D)              (None, 3, 18, 64)    18496       conv2d_14[0][0]                  
__________________________________________________________________________________________________
flatten_8 (Flatten)             (None, 3456)         0           conv2d_15[0][0]                  
____________________________________________________________________________________________

In [73]:
# Build the decoder
latent_inputs = keras.Input(shape=(latent_dim,))
x = layers.Dense(3 * 18 * 64, activation="relu")(latent_inputs)
x = layers.Reshape((3, 18, 64))(x)
x = layers.Conv2DTranspose(filters= 64, kernel_size= 3, activation="relu", strides=2, padding="same")(x)
x = layers.Conv2DTranspose(filters= 32, kernel_size= 3, activation="relu", strides=2, padding="same")(x)
decoder_outputs = layers.Conv2DTranspose(1, 3, activation="sigmoid", padding="same")(x)
decoder = keras.Model(latent_inputs, decoder_outputs, name="decoder")
decoder.summary()

Model: "decoder"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_18 (InputLayer)        [(None, 32)]              0         
_________________________________________________________________
dense_17 (Dense)             (None, 3456)              114048    
_________________________________________________________________
reshape_8 (Reshape)          (None, 3, 18, 64)         0         
_________________________________________________________________
conv2d_transpose_21 (Conv2DT (None, 6, 36, 64)         36928     
_________________________________________________________________
conv2d_transpose_22 (Conv2DT (None, 12, 72, 32)        18464     
_________________________________________________________________
conv2d_transpose_23 (Conv2DT (None, 12, 72, 1)         289       
Total params: 169,729
Trainable params: 169,729
Non-trainable params: 0
_____________________________________________________

In [245]:
# Define the VAE as a Model with a custom train_step
class VAE(keras.Model):
    def __init__(self, encoder, decoder, **kwargs):
        super(VAE, self).__init__(**kwargs)
        self.encoder = encoder
        self.decoder = decoder

    def train_step(self, data):
        if isinstance(data, tuple):
            data = data[0]
        with tf.GradientTape() as tape:
            z_mean, z_log_var, z = encoder(data)
            reconstruction = decoder(z)
            reconstruction_loss = tf.reduce_mean(
                keras.losses.mean_squared_error(data, reconstruction)
            )
            reconstruction_loss *= 12 * 72
            kl_loss = 1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var)
            kl_loss = tf.reduce_mean(kl_loss)
            kl_loss *= -0.5
            total_loss = reconstruction_loss + kl_loss
        grads = tape.gradient(total_loss, self.trainable_weights)
        self.optimizer.apply_gradients(zip(grads, self.trainable_weights))
        return {
            "loss": total_loss,
            "reconstruction_loss": reconstruction_loss,
            "kl_loss": kl_loss,
        }

In [287]:
vae = VAE(encoder, decoder)
vae.compile(optimizer=keras.optimizers.Adam())
vae.fit(reshapeData, epochs=50, batch_size=64)

Epoch 1/50
792/792 [==============================] - 32s 41ms/step - loss: 18.0740 - reconstruction_loss: 17.6169 - kl_loss: 0.4571
Epoch 2/50
792/792 [==============================] - 34s 43ms/step - loss: 17.6948 - reconstruction_loss: 17.3001 - kl_loss: 0.3946
Epoch 3/50
792/792 [==============================] - 36s 45ms/step - loss: 17.5447 - reconstruction_loss: 17.1862 - kl_loss: 0.3585
Epoch 4/50
792/792 [==============================] - 40s 51ms/step - loss: 17.4276 - reconstruction_loss: 17.1040 - kl_loss: 0.3236
Epoch 5/50
792/792 [==============================] - 36s 46ms/step - loss: 17.3525 - reconstruction_loss: 17.0408 - kl_loss: 0.3117
Epoch 6/50
792/792 [==============================] - 36s 45ms/step - loss: 17.3350 - reconstruction_loss: 17.0348 - kl_loss: 0.3002
Epoch 7/50
792/792 [==============================] - 37s 47ms/step - loss: 17.3511 - reconstruction_loss: 17.0646 - kl_loss: 0.2865
Epoch 8/50
792/792 [==============================] - 34s 43ms/step -

In [288]:
encDec = decoder.predict(encoder.predict(reshapeData))

In [302]:
encDec.shape

(50625, 12, 72, 1)

In [314]:
# Putting the matrix back together
result = np.array([[0 for x in range(16200)] for x in range(2700)], dtype = 'float32')
encDec = encDec.reshape(reshapeData.shape[0], reshapeData.shape[1], reshapeData.shape[2])

for i in range(0, 2700, 12):   # cells
    for j in range(0, 16200, 72):   # genes
        x = ( 225*(i/12) ) + ( (j/72)+1 )
        if x%100 == 0:
            print("Starting run =", x)
        sect = encDec[int(x-1),:,:]
        result[i:(i+12) , j:(j+72)] = sect
    
result.shape

Starting run = 100.0
Starting run = 200.0
Starting run = 300.0
Starting run = 400.0
Starting run = 500.0
Starting run = 600.0
Starting run = 700.0
Starting run = 800.0
Starting run = 900.0
Starting run = 1000.0
Starting run = 1100.0
Starting run = 1200.0
Starting run = 1300.0
Starting run = 1400.0
Starting run = 1500.0
Starting run = 1600.0
Starting run = 1700.0
Starting run = 1800.0
Starting run = 1900.0
Starting run = 2000.0
Starting run = 2100.0
Starting run = 2200.0
Starting run = 2300.0
Starting run = 2400.0
Starting run = 2500.0
Starting run = 2600.0
Starting run = 2700.0
Starting run = 2800.0
Starting run = 2900.0
Starting run = 3000.0
Starting run = 3100.0
Starting run = 3200.0
Starting run = 3300.0
Starting run = 3400.0
Starting run = 3500.0
Starting run = 3600.0
Starting run = 3700.0
Starting run = 3800.0
Starting run = 3900.0
Starting run = 4000.0
Starting run = 4100.0
Starting run = 4200.0
Starting run = 4300.0
Starting run = 4400.0
Starting run = 4500.0
Starting run = 4600

(2700, 16200)

In [332]:
result_df = pd.DataFrame(result)
result_df

sum(result_df.sum(axis = 1) != 397.483429)
# Outputed matrix filled with all 0.024536
# something went wrong...

,0,1,2,3,4,5,6,7,8,9,...,16190,16191,16192,16193,16194,16195,16196,16197,16198,16199
0,0.024536,0.024536,0.024536,0.024536,0.024536,0.024536,0.024536,0.024536,0.024536,0.024536,...,0.024536,0.024536,0.024536,0.024536,0.024536,0.024536,0.024536,0.024536,0.024536,0.024536
1,0.024536,0.024536,0.024536,0.024536,0.024536,0.024536,0.024536,0.024536,0.024536,0.024536,...,0.024536,0.024536,0.024536,0.024536,0.024536,0.024536,0.024536,0.024536,0.024536,0.024536
2,0.024536,0.024536,0.024536,0.024536,0.024536,0.024536,0.024536,0.024536,0.024536,0.024536,...,0.024536,0.024536,0.024536,0.024536,0.024536,0.024536,0.024536,0.024536,0.024536,0.024536
3,0.024536,0.024536,0.024536,0.024536,0.024536,0.024536,0.024536,0.024536,0.024536,0.024536,...,0.024536,0.024536,0.024536,0.024536,0.024536,0.024536,0.024536,0.024536,0.024536,0.024536
4,0.024536,0.024536,0.024536,0.024536,0.024536,0.024536,0.024536,0.024536,0.024536,0.024536,...,0.024536,0.024536,0.024536,0.024536,0.024536,0.024536,0.024536,0.024536,0.024536,0.024536
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2695,0.024536,0.024536,0.024536,0.024536,0.024536,0.024536,0.024536,0.024536,0.024536,0.024536,...,0.024536,0.024536,0.024536,0.024536,0.024536,0.024536,0.024536,0.024536,0.024536,0.024536
2696,0.024536,0.024536,0.024536,0.024536,0.024536,0.024536,0.024536,0.024536,0.024536,0.024536,...,0.024536,0.024536,0.024536,0.024536,0.024536,0.024536,0.024536,0.024536,0.024536,0.024536
2697,0.024536,0.024536,0.024536,0.024536,0.024536,0.024536,0.024536,0.024536,0.024536,0.024536,...,0.024536,0.024536,0.024536,0.024536,0.024536,0.024536,0.024536,0.024536,0.024536,0.024536
2698,0.024536,0.024536,0.024536,0.024536,0.024536,0.024536,0.024536,0.024536,0.024536,0.024536,...,0.024536,0.024536,0.024536,0.024536,0.024536,0.024536,0.024536,0.024536,0.024536,0.024536


0

# Trying Autoencoder with Dense Layers (not Conv.)
Since the conv failed so bad. Want to see if beacause of Conv or not...

In [25]:
model = Sequential()
fullData = np.asarray(dataNorm)
n_genes = fullData.shape[1]

#add model layers
model.add(layers.Input(shape= n_genes))
model.add(layers.Dense(8192, activation='relu'))
model.add(layers.Dense(4096, activation='relu'))
model.add(layers.Dense(1024, activation='relu'))
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(32, activation='relu'))
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(1024, activation='relu'))
model.add(layers.Dense(4096, activation='relu'))
model.add(layers.Dense(8192, activation='relu'))
model.add(layers.Dense(n_genes, activation='relu'))

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_9 (Dense)              (None, 8192)              136273920 
_________________________________________________________________
dense_10 (Dense)             (None, 4096)              33558528  
_________________________________________________________________
dense_11 (Dense)             (None, 1024)              4195328   
_________________________________________________________________
dense_12 (Dense)             (None, 512)               524800    
_________________________________________________________________
dense_13 (Dense)             (None, 32)                16416     
_________________________________________________________________
dense_14 (Dense)             (None, 512)               16896     
_________________________________________________________________
dense_15 (Dense)             (None, 1024)             

In [348]:
# Train test split (random train = 90%)
# trainProp = 0.9
# n_cells = fullData.shape[0]

# train_idx = np.random.choice(n_cells, int(n_cells * trainProp), replace= False)
# test_idx = np.array(list(set(range(n_cells)) - set(train_idx)))
# trainAE = fullData[train_idx, :]
# testAE = fullData[test_idx, :]

In [26]:
def nzMSE(y_true, y_pred): 
    omega = tf.sign(y_true)  # 0 if 0, 1 if > 0
    mse_nz = tf.reduce_mean(tf.multiply(tf.pow( (y_pred - y_true), 2), omega))
    return mse_nz

model.compile(optimizer='adam', 
              loss= nzMSE)

In [60]:
# Train model
AEresults = model.fit(x= fullData, 
                      y= fullData, 
                      epochs= 100, 
                      batch_size= 128)

In [ ]:
predicted = pd.DataFrame(model.predict(fullData))
predicted
# Notes on using MSE...
    # At least it's not all zeros this time, but still needs some work
    # Need to build a loss function to penalize for zeros 

# Now added loss function described in paper (nonzero MSE)...
    # Did pretty well!!
    # 95% zeros --> 28% zeros

In [ ]:
data.eq(0).values.sum()/(2700*16634)
predicted.eq(0).values.sum()/(2700*16634)


# Testing dropout layers

In [62]:
modelD = Sequential()
fullData = np.asarray(dataNorm)
n_genes = fullData.shape[1]

#add model layers
modelD.add(layers.Input(shape= n_genes))
modelD.add(layers.Dropout(0.2))
modelD.add(layers.Dense(8192, activation='relu'))
modelD.add(layers.Dropout(0.2))
modelD.add(layers.Dense(4096, activation='relu'))
modelD.add(layers.Dropout(0.2))
modelD.add(layers.Dense(256, activation='relu'))

modelD.add(layers.Dense(32, activation='relu'))

modelD.add(layers.Dense(256, activation='relu'))
modelD.add(layers.Dropout(0.2))
modelD.add(layers.Dense(4096, activation='relu'))
modelD.add(layers.Dropout(0.2))
modelD.add(layers.Dense(8192, activation='relu'))
modelD.add(layers.Dropout(0.2))
modelD.add(layers.Dense(n_genes, activation='relu'))

modelD.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dropout_16 (Dropout)         (None, 16634)             0         
_________________________________________________________________
dense_40 (Dense)             (None, 8192)              136273920 
_________________________________________________________________
dropout_17 (Dropout)         (None, 8192)              0         
_________________________________________________________________
dense_41 (Dense)             (None, 4096)              33558528  
_________________________________________________________________
dropout_18 (Dropout)         (None, 4096)              0         
_________________________________________________________________
dense_42 (Dense)             (None, 256)               1048832   
_________________________________________________________________
dense_43 (Dense)             (None, 32)               

In [64]:
modelD.compile(optimizer='adam', 
               loss='mean_squared_error')

In [66]:
# Train model
dropoutAdded = modelD.fit(x= fullData, 
                          y= fullData, 
                          epochs= 10, 
                          batch_size= 64)

Epoch 1/10
43/43 [==============================] - 48s 1s/step - loss: 0.0426
Epoch 2/10
43/43 [==============================] - 50s 1s/step - loss: 0.0284
Epoch 3/10
43/43 [==============================] - 48s 1s/step - loss: 0.0283
Epoch 4/10
43/43 [==============================] - 48s 1s/step - loss: 0.0281
Epoch 5/10
43/43 [==============================] - 48s 1s/step - loss: 0.0271
Epoch 6/10
43/43 [==============================] - 48s 1s/step - loss: 0.0263
Epoch 7/10
43/43 [==============================] - 48s 1s/step - loss: 0.0261
Epoch 8/10
43/43 [==============================] - 48s 1s/step - loss: 0.0260
Epoch 9/10
43/43 [==============================] - 48s 1s/step - loss: 0.0257
Epoch 10/10
43/43 [==============================] - 48s 1s/step - loss: 0.0254


In [67]:
predDropout = pd.DataFrame(modelD.predict(fullData))
predDropout

,0,1,2,3,4,5,6,7,8,9,...,16624,16625,16626,16627,16628,16629,16630,16631,16632,16633
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.503495,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.360314,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.479689,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.253325,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.250109,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2695,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.214159,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2696,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.431963,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2697,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.357407,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2698,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.258380,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [68]:
data.eq(0).values.sum()/(2700*16634)
predDropout.eq(0).values.sum()/(2700*16634)
# Actually made it worse

0.9490805534402985

0.9433882186864031

# Finding Number of Significant PCs 
Maybe that gives appropriate size of latent space (coding size)

In [46]:
from matplotlib import*
import matplotlib.pyplot as plt
from matplotlib.cm import register_cmap
from scipy import stats
from sklearn.decomposition import PCA as sklearnPCA
import seaborn

In [49]:
X_std = StandardScaler().fit_transform(fullData)
print('NumPy covariance matrix: \n%s' %np.cov(X_std.T))

NumPy covariance matrix: 
[[ 1.00037053e+00 -1.88282065e-03 -2.26857835e-03 ... -1.09819843e-03
  -1.13005401e-02  3.46964770e-02]
 [-1.88282065e-03  1.00037054e+00 -1.31268066e-03 ... -6.35456933e-04
  -6.53889713e-03 -5.10513164e-03]
 [-2.26857835e-03 -1.31268066e-03  1.00037046e+00 ... -7.65651175e-04
   2.42811796e-03 -6.15108567e-03]
 ...
 [-1.09819843e-03 -6.35456933e-04 -7.65651175e-04 ...  1.00037049e+00
  -3.81396209e-03 -2.97768540e-03]
 [-1.13005401e-02 -6.53889713e-03  2.42811796e-03 ... -3.81396209e-03
   1.00037052e+00  3.17662551e-03]
 [ 3.46964770e-02 -5.10513164e-03 -6.15108567e-03 ... -2.97768540e-03
   3.17662551e-03  1.00037051e+00]]


In [54]:
cov_mat = np.cov(X_std.T)
eig_vals, eig_vecs = np.linalg.eig(cov_mat)
print('Eigenvectors \n%s' %eig_vecs)
print('\nEigenvalues \n%s' %eig_vals)

In [ ]:
# Visually confirm that the list is correctly sorted by decreasing eigenvalues
eig_pairs = [(np.abs(eig_vals[i]), eig_vecs[:,i]) for i in range(len(eig_vals))]
print('Eigenvalues in descending order:')
for i in eig_pairs:
    print(i[0])

In [ ]:
pca = PCA(n_components=)
pca.fit_transform(df1)
print pca.explained_variance_ratio_ 

In [ ]:
#Explained variance
pca = PCA().fit(X_std)
plt.plot(np.cumsum(pca.explained_variance_ratio_))
plt.xlabel('number of components')
plt.ylabel('cumulative explained variance')
plt.show()

# Implementing Variational Autoencoder


In [47]:
# class Sampling(layers.Layer): 
#     def call(self, inputs):
#         mean, log_var = inputs
#         return K.random_normal(tf.shape(log_var)) * K.exp(log_var / 2) + mean

In [85]:
# Create a sampling layer
class Sampling(layers.Layer):
    """Uses (z_mean, z_log_var) to sample z, the vector encoding a digit."""

    def call(self, inputs):
        z_mean, z_log_var = inputs
        batch = tf.shape(z_mean)[0]
        dim = tf.shape(z_mean)[1]
        epsilon = tf.keras.backend.random_normal(shape=(batch, dim))
        return z_mean + tf.exp(0.5 * z_log_var) * epsilon

In [86]:
fullData = np.asarray(dataNorm)

In [87]:
codings_size = 32

inputs = layers.Input(shape=16634)
z = layers.Dense(8000)(inputs)
z = layers.Dense(4000, activation= "relu")(z)
z = layers.Dense(1000, activation="relu")(z)
z = layers.Dense(256, activation="relu")(z) 

codings_mean = layers.Dense(codings_size)(z) # μ 
codings_log_var = layers.Dense(codings_size)(z) # γ 
codings = Sampling()([codings_mean, codings_log_var]) 

variational_encoder = Model(
    inputs=[inputs], outputs=[codings_mean, codings_log_var, codings])
variational_encoder.summary()

Model: "model_17"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_14 (InputLayer)           [(None, 16634)]      0                                            
__________________________________________________________________________________________________
dense_79 (Dense)                (None, 8000)         133080000   input_14[0][0]                   
__________________________________________________________________________________________________
dense_80 (Dense)                (None, 4000)         32004000    dense_79[0][0]                   
__________________________________________________________________________________________________
dense_81 (Dense)                (None, 1000)         4001000     dense_80[0][0]                   
___________________________________________________________________________________________

In [88]:
decoder_inputs = layers.Input(shape=[codings_size])

x = layers.Dense(256, activation="relu")(decoder_inputs)
x = layers.Dense(1000, activation="relu")(x)
x = layers.Dense(4000, activation="relu")(x)
x = layers.Dense(8000, activation="relu")(x)
outputs = layers.Dense(16634, activation= "relu")(x)

variational_decoder = Model(inputs=[decoder_inputs], outputs=[outputs])
variational_decoder.summary()

Model: "model_18"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_15 (InputLayer)        [(None, 32)]              0         
_________________________________________________________________
dense_85 (Dense)             (None, 256)               8448      
_________________________________________________________________
dense_86 (Dense)             (None, 1000)              257000    
_________________________________________________________________
dense_87 (Dense)             (None, 4000)              4004000   
_________________________________________________________________
dense_88 (Dense)             (None, 8000)              32008000  
_________________________________________________________________
dense_89 (Dense)             (None, 16634)             133088634 
Total params: 169,366,082
Trainable params: 169,366,082
Non-trainable params: 0
____________________________________________

In [89]:
# _, _, codings = variational_encoder(inputs)
# reconstructions = variational_decoder(codings)
# variational_ae = Model(inputs=[inputs], outputs=[reconstructions])
# variational_ae.summary()

In [97]:
# Define the VAE as a Model with a custom train_step
class VAE(keras.Model):
    def __init__(self, variational_encoder, variational_decoder, **kwargs):
        super(VAE, self).__init__(**kwargs)
        self.variational_encoder = variational_encoder
        self.variational_decoder = variational_decoder

    def train_step(self, data):
        if isinstance(data, tuple):
            data = data[0]
        with tf.GradientTape() as tape:
            
            _, _, codings = variational_encoder(data)
            reconstruction = variational_decoder(codings)
            
            omega = tf.sign(data)  # 0 if 0, 1 if > 0
            reconstruction_loss = tf.reduce_mean(tf.multiply(tf.pow( (data - reconstruction), 2), omega))
    
            kl_loss = 1 + codings_log_var - tf.square(codings_mean) - tf.exp(codings_log_var)
            kl_loss = tf.reduce_mean(kl_loss)
            kl_loss *= -0.5
            total_loss = reconstruction_loss + kl_loss
        
        grads = tape.gradient(total_loss, self.trainable_weights)
        self.optimizer.apply_gradients(zip(grads, self.trainable_weights))
        tf.config.experimental_run_functions_eagerly(True)
        return {
            "loss": total_loss,
            "reconstruction_loss": reconstruction_loss,
            "kl_loss": kl_loss,
        }

In [98]:
variational_ae = VAE(variational_encoder, variational_decoder)
variational_ae.compile(optimizer= 'adam')

In [ ]:
# Train model
VAEresults = variational_ae.fit(fullData, 
                                epochs= 10, 
                                batch_size= 256)

So this approach didn't work (too large of a size). Trying a different approach...

# Variational Autoencoder (again)
http://louistiao.me/posts/implementing-variational-autoencoders-in-keras-beyond-the-quickstart-tutorial/

In [8]:
# Define variables
fullData = np.asarray(dataNorm)[:,:1000]
original_dim = 1000
latent_dim = 16
batch_size = 64
epochs = 10
epsilon_std = 1.0

In [9]:
def nzMSE(y_true, y_pred):
    """ MSE for nonzero values. """
    omega = tf.sign(y_true)  # 0 if 0, 1 if > 0
    mse_nz = tf.reduce_mean(tf.multiply(tf.pow( (y_pred - y_true), 2), omega))
    return mse_nz

In [10]:
class KLDivergenceLayer(layers.Layer):

    """ Identity transform layer that adds KL divergence
    to the final model loss.
    """

    def __init__(self, *args, **kwargs):
        self.is_placeholder = True
        super(KLDivergenceLayer, self).__init__(*args, **kwargs)

    def call(self, inputs):

        mu, log_var = inputs

        kl_batch = - .5 * K.sum(1 + log_var -
                                K.square(mu) -
                                K.exp(log_var), axis=-1)

        self.add_loss(K.mean(kl_batch), inputs=inputs)

        return inputs

In [11]:
# Creating the Decoder 
decoder = Sequential([
    layers.Dense(128, input_dim=latent_dim, activation='relu'),
    layers.Dense(512, activation="relu"),
    layers.Dense(original_dim, activation= "relu")
])

decoder.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 128)               2176      
_________________________________________________________________
dense_1 (Dense)              (None, 512)               66048     
_________________________________________________________________
dense_2 (Dense)              (None, 1000)              513000    
Total params: 581,224
Trainable params: 581,224
Non-trainable params: 0
_________________________________________________________________


In [17]:
x = layers.Input(shape= (original_dim,))
xh = layers.Dense(512, activation="relu")(x)
h = layers.Dense(128, activation="relu")(xh)

In [18]:
z_mu = layers.Dense(latent_dim)(h)
z_log_var = layers.Dense(latent_dim)(h)

z_mu, z_log_var = KLDivergenceLayer()([z_mu, z_log_var])
z_sigma = layers.Lambda(lambda t: K.exp(.5*t))(z_log_var)

In [19]:
eps = layers.Input(tensor=K.random_normal(stddev=epsilon_std,
                                          shape=(K.shape(x)[0], latent_dim)))
z_eps = layers.Multiply()([z_sigma, eps])
z = layers.Add()([z_mu, z_eps])

x_pred = decoder(z)

In [20]:
vae = Model(inputs=[x, eps], outputs=x_pred)
vae.compile(optimizer= 'adam', loss= nzMSE)
vae.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 1000)]       0                                            
__________________________________________________________________________________________________
dense_5 (Dense)                 (None, 512)          512512      input_3[0][0]                    
__________________________________________________________________________________________________
dense_6 (Dense)                 (None, 128)          65664       dense_5[0][0]                    
__________________________________________________________________________________________________
dense_7 (Dense)                 (None, 16)           2064        dense_6[0][0]                    
______________________________________________________________________________________________

In [24]:
vae.fit(fullData,
        fullData,
        epochs=epochs,
        batch_size=batch_size)

Epoch 1/10


AssertionError: in user code:

    /home/ahmadazim/anaconda3/envs/tflow/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:571 train_function  *
        outputs = self.distribute_strategy.run(
    /home/ahmadazim/anaconda3/envs/tflow/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:951 run  **
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /home/ahmadazim/anaconda3/envs/tflow/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:2290 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /home/ahmadazim/anaconda3/envs/tflow/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:2649 _call_for_each_replica
        return fn(*args, **kwargs)
    /home/ahmadazim/anaconda3/envs/tflow/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:531 train_step  **
        y_pred = self(x, training=True)
    /home/ahmadazim/ana